In [1]:
!pip install torchmetrics
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor,Normalize,Compose,Resize
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import itertools
import time
import torch.utils.data as data_utils
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinaryPrecision
from keras.datasets import cifar10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 9.1 MB/s eta 0:00:00


In [2]:
transform = Compose(
    [ToTensor(),Resize(224),Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = CIFAR10(root='data/', download=True, transform=transform)
test_dataset = CIFAR10(root='data/', train=False, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


In [3]:
classes=dataset.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [4]:
train_loader = DataLoader(dataset, 32, shuffle=True)
test_loader = DataLoader(test_dataset, 64, shuffle=False)

In [5]:
!pip install transformers
from transformers import BeitFeatureExtractor, BeitForImageClassification
import torch

model = BeitForImageClassification.from_pretrained("jadohu/BEiT-finetuned")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.1 MB/s eta 0:00:00


Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [6]:
pars=sum(p.numel() for p in model.parameters())
pars

85769674

In [7]:
model

BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (relative_position_bias): BeitRelativePositionBias()
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (int

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(device)
model.to(device)

cuda:0


BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (relative_position_bias): BeitRelativePositionBias()
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (int

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=0.00005)
metricF1=BinaryF1Score()
metricAcc=BinaryAccuracy()
metricPre=BinaryPrecision()
softmax = nn.Softmax(dim=1)
criterion.to(device)

CrossEntropyLoss()

In [14]:
def train_loop(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses=np.zeros(epochs)
  valid_losses=np.zeros(epochs)
  Accuracy=np.zeros(epochs)

  for i in range(epochs):
    train_loss=[]
    valid_loss=[]
    f1 = []
    accuracy = []
    precision=[]

    model.train()
    for _, data in enumerate(train_loader):
      datas, targets=data
      datas, targets= datas.to(device), targets.to(device)
      datas=datas.squeeze(1)
      optimizer.zero_grad()
      outputs=model(pixel_values=datas)
      logits = outputs.logits
      loss=criterion(logits, targets)
      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())
    train_loss=np.mean(train_loss)
    # with torch.no_grad():
    #   model.eval()
    #   for _, data in enumerate(valid_loader):
    #     datas, targets=data
    #     datas, targets= datas.to(device), targets.to(device)
    #     datas=datas.squeeze(1)
    #     outputs=model(pixel_values=datas)
    #     logits = outputs.logits
    #     loss=criterion(logits, targets)
    #     valid_loss.append(loss.item())
    #     outputs=softmax(outputs)
    #     yy = torch.zeros(outputs.shape[0], 10)
    #     for i in range(outputs.shape[0]):
    #       yy[i][targets[i]] = 1.
    #     f1.append(metricF1(outputs.cpu(),yy.cpu()))
    #     accuracy.append(metricAcc(outputs.cpu(),yy.cpu()))
    #     precision.append(metricPre(outputs.cpu(),yy.cpu()))

    # valid_loss=np.mean(valid_loss)
    # f1=np.mean(f1)
    # accuracy=np.mean(accuracy)
    # precisionn=np.mean(precision)

    train_losses[i]=train_loss
    # valid_losses[i]=valid_loss
    # Accuracy[i]=accuracy

    print(f'epoch {i+1}/{epochs}, train loss: {train_loss:.4f}')
      #  / validation loss: {valid_loss:.4f}')
    # print(f'epoch {i+1}/{epochs}, f1: {f1:.4f}, \
    #       accuracy: {accuracy:.4f}, \
    #       precision: {precisionn:.4f}')
    
  return train_losses, valid_losses, Accuracy

In [15]:
train_losses, test_losses, accuracy = train_loop(model, criterion, optimizer,
                                                        train_loader,test_loader, epochs=5)

epoch 1/5, train loss: 0.0533
epoch 2/5, train loss: 0.0443


KeyboardInterrupt: ignored

In [32]:
# valid_losses=np.zeros(epochs)
# Accuracy=np.zeros(epochs)
valid_loss=[]
f1 = []
accuracy = []
precision=[]
with torch.no_grad():
      # model.eval()
      for _, data in enumerate(train_loader):
        datas, targets=data
        datas, targets= datas.to(device), targets.to(device)
        datas=datas.squeeze(1)
        outputs=model(pixel_values=datas)
        logits = outputs.logits
        loss=criterion(logits, targets)
        valid_loss.append(loss.item())
        outputs=softmax(logits)
        yy = torch.zeros(outputs.shape[0], 10)
        for i in range(outputs.shape[0]):
          yy[i][targets[i]] = 1.
        f1.append(metricF1(outputs.cpu(),yy.cpu()))
        accuracy.append(metricAcc(outputs.cpu(),yy.cpu()))
        precision.append(metricPre(outputs.cpu(),yy.cpu()))

valid_loss=np.mean(valid_loss)
f1=np.mean(f1)
accuracy=np.mean(accuracy)
precision=np.mean(precision)
print(f'validation loss: {valid_loss:.4f}')
print(f'epoch {i+1}/{epochs}, f1: {f1:.4f}, \
      accuracy: {accuracy:.4f}, \
      precision: {precisionn:.4f}')

KeyboardInterrupt: ignored

In [33]:
valid_losses=np.mean(valid_loss)
f11=np.mean(f1)
accuraccy=np.mean(accuracy)
precisionn=np.mean(precision)
print(f'validation loss: {valid_losses:.4f}')
print(f'f1: {f11:.4f}, \
      accuracy: {accuraccy:.4f}, \
      precision: {precisionn:.4f}')

validation loss: 0.0308
f1: 0.9894,       accuracy: 0.9979,       precision: 0.9904


In [ ]:
plt.plot(valid_loss)
plt.title("test loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
plt.plot(accuracy)
plt.title("accuracy")
plt.xlabel("epoch")
plt.ylabel("acc")
plt.show()

In [36]:
def plot_conf_matrix(cm,classes, nomalize=False, cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation='nearest',cmap=cmap)
  plt.colorbar()
  tick_marks=np.arange(len(classes))
  plt.xticks(tick_marks,classes, rotation=45)
  plt.yticks(tick_marks,classes)
  fmt='.2f'
  thresh=cm.max() / 2
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
      plt.text(j,i,format(cm[i,j],fmt),
               horizontalalignment="center",
               color="white" if cm[i,j] > thresh else "black") 
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()



In [57]:
p_test=np.array([])
# x, data=enumerate(train_loader)
# datas, targets= data
# train_loader.
  # datas, targets= datas.to(device), targets.to(device)
  # datas=datas.squeeze(1)
  # outputs=model(datas)
out=outputs.logits
print(len(out))
print(len(dataset.targets))
print(len(yy))
_,prediction=torch.max(out,1)
p_test=np.concatenate((p_test,prediction.cpu().detach().numpy()))
print(len(p_test))
cm=confusion_matrix(np.array(targets),p_test)
plot_conf_matrix(cm,list(range(10)))

ValueError: ignored